In [ ]:
import cv2
import os

img_dir = "data/images"
canny_dir = "data/canny"
os.makedirs(canny_dir, exist_ok=True)

for fname in os.listdir(img_dir):
    # print(fname)
    img = cv2.imread(os.path.join(img_dir, fname))
    # print(img)
    edges = cv2.Canny(img, 100, 200)
    cv2.imwrite(os.path.join(canny_dir, fname), edges)

Elets.png
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
Iasi.png
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
 

In [ ]:
accelerate launch train_controlnet_lora.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --controlnet_model_name_or_path="lllyasviel/sd-controlnet-canny" \
  --train_data_dir="data" \
  --output_dir="output" \
  --caption_file="captions.json" \
  --resolution=512 \
  --train_batch_size=1 \
  --learning_rate=1e-5 \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --lr_scheduler="constant" \
  --num_train_epochs=100 \
  --validation_prompt="2 адм, 6 РВС 10к м, 1 РВС 500 м, 4 РВС 50 м, 2 жд" \
  --tracker_project_name="building-schemes"


^C


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
C:\Users\nadyl\AppData\Local\Programs\Python\Python310\python.exe: can't open file 'c:\\Users\\nadyl\\Documents\\GitHub\\generation-of-building-diagrams\\Attempt 4\\train_controlnet_lora.py': [Errno 2] No such file or directory
Traceback (most recent call last):
  File "C:\Users\nadyl\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\nadyl\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\nadyl\Documents\GitHub\generation-of-building-diagr

Проверяем все описание на адекватное количество токенов, чтобы не потерять никакой информации

In [33]:
from transformers import CLIPTokenizer

tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")

text = "4 адм, 6 РВС 2к м, 8 ГВС 75 м, 8 ГВС 50 м, 2 жд"

tokens = tokenizer(text, truncation=False, padding=False)
num_tokens = len(tokens['input_ids'])

print(f"Количество токенов: {num_tokens}")

if num_tokens > 77:
    print("Текст слишком длинный и будет обрезан.")
else:
    print("Текст подходит по длине.")


Количество токенов: 46
Текст подходит по длине.
